## 1. Perceptron

<img src="./img/perceptron-6168423.jpg" alt="drawing" width="650"/>

Empezamos cargando librerías

In [91]:
import numpy as np
import pandas as pd
import seaborn as sns

Cargamos datos. Utilizaremos el dataset de pinguinos de seaborn

In [92]:
df = sns.load_dataset("penguins")

# Limpiamos un poco los datos
df.dropna(inplace=True)
cleanup_nums = {"species": {"Adelie": 0,
                            "Chinstrap": 1,
                            "Gentoo": 2},
               "sex": {"Male": 0,
                       "Female": 1}}
df.replace(cleanup_nums, inplace=True)
df = pd.get_dummies(df)

df.head()

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,island_Biscoe,island_Dream,island_Torgersen,sex_FEMALE,sex_MALE
0,0,39.1,18.7,181.0,3750.0,0,0,1,0,1
1,0,39.5,17.4,186.0,3800.0,0,0,1,1,0
2,0,40.3,18.0,195.0,3250.0,0,0,1,1,0
4,0,36.7,19.3,193.0,3450.0,0,0,1,1,0
5,0,39.3,20.6,190.0,3650.0,0,0,1,0,1


Dividimos en train test

In [93]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, 1:]
y = df.iloc[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [94]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(266, 9)
(67, 9)
(266,)
(67,)


Vamos a probar un Perceptrón

In [95]:
from sklearn.linear_model import Perceptron

per_clf = Perceptron()
per_clf.fit(X_train, y_train)
per_clf.score(X_test, y_test)

0.34328358208955223

In [96]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train, y_train)
log_reg.score(X_test, y_test)

0.9850746268656716

Probemos a estandarizar:

In [97]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_s = sc.transform(X_train)
X_test_s = sc.transform(X_test)

per_clf = Perceptron()
per_clf.fit(X_train_s, y_train)
per_clf.score(X_test_s, y_test)

1.0

In [98]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train_s, y_train)
log_reg.score(X_test_s, y_test)

1.0

En este caso, en ambos modelos, parece que la estandarización es la clave, y es que ambos modelos son muy dependientes de las escalas de los datos.

Aun así, si nos fijamos en el comportamiento base del perceptrón, parece que no es muy bueno (pese a que con ayuda del StandardScaler mejora mucho). Y eso que estamos hablando de un modelo muy sencillito. Por lo tanto, parece razonable estudiar configuraciones más complejas:

## 2. Multi Layer Perceptron

In [99]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter = 500)
mlp.fit(X_train, y_train)

print(mlp.score(X_train, y_train))
print(mlp.score(X_test, y_test))

0.5676691729323309
0.5373134328358209


Probemos otra configuración. Es posible crear una red neuronal desde la propia función de MLPClassifier()

In [100]:
mlp = MLPClassifier(max_iter = 500,
                   activation='tanh',
                   hidden_layer_sizes = (150, 150, 150))

mlp.fit(X_train, y_train)

print(mlp.score(X_train, y_train))
print(mlp.score(X_test, y_test))

0.6766917293233082
0.6716417910447762


Bueno, parece que nuestra red de perceptrones mejora mucho el perceptrón por sí solo, pero seguimos obteniendo algo que dista mucho de lo que queremos. Y es que, al igual que acabamos de ver para el perceptrón, las redes multicapa de perceptrones utilizan descenso del gradiente, y por tanto son muy sensibles al escalado.

Estandarizamos para el siguiente ejemplo:

In [105]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_s = sc.transform(X_train)
X_test_s = sc.transform(X_test)

per_clf = Perceptron()
per_clf.fit(X_train_s, y_train)
print(per_clf.score(X_train_s, y_train))
print(per_clf.score(X_test_s, y_test))

1.0
1.0


In [106]:
log_reg = LogisticRegression(max_iter=500)
log_reg.fit(X_train_s, y_train)
print(log_reg.score(X_train_s, y_train))
print(log_reg.score(X_test_s, y_test))

1.0
1.0


In [107]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scal =scaler.transform(X_train)
X_test_scal =scaler.transform(X_test)

mlp = MLPClassifier(max_iter=500)
mlp.fit(X_train_scal, y_train)
print(mlp.score(X_train_scal, y_train))
print(mlp.score(X_test_scal, y_test))

1.0
1.0


Evidentemente, como estamos con unos datos muy preparados para este ejemplo, al estandarizar nos ofrecen un resultado perfecto, como podíamos esperar tras ver el ejemplo anterior.

De este modo, se pone de manifiesto la importancia del escalado en este tipo de problemas, ya que son muy dependientes de las relaciones entre variables. Por ello, será muy importante trabajar con el StandardScaler cuando hablemos de modelos lineales, ya que tanto el ``Perceptrón``, como la ``Regresión Logística`` o la configuración ``MLP`` consiguen un resultado muy bueno tras su aplicación.

Pero bueno, realmente no hemos visto nada que nos mejore una barbaridad, así que hagamos un ejercicio donde pongamos las cosas (un poquito) más difíciles a nuestro modelo:

### Ejercicio

En este caso, vamos a ver cómo se comportan nuestros modelos con la clasificación de medicamentos. Para ello, leeremos un dataset con algunas columnas categóricas, que deberemos convertir a numéricas de la forma que creamos conveniente.

Utiliza un reparto train/test de 70/30.

In [104]:
df = pd.read_csv("../drug200.csv")